<a href="https://colab.research.google.com/github/gitmystuff/INFO4080/blob/main/Week_05-Introduction_Section/WikiScrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WikiScrape



## Mount Drive

## Wikipedia API

If you intend to do any scraping projects or automated requests, consider alternatives such as Pywikipediabot or MediaWiki API, which has other superior features.

* wikipedia.search('keywords', results=2)
* wikipedia.suggest('keyword')
* wikipedia.summary('keywords', sentences=2)
* wikipedia.page('keywords')
* wikipedia.page('keywords').content
* wikipedia.page('keywords').references
* wikipedia.page('keywords').title
* wikipedia.page('keywords').url
* wikipedia.page('keywords').categories
* wikipedia.page('keywords').content
* wikipedia.page('keywords').links
* wikipedia.geosearch(33.2075, 97.1526)
* wikipedia.set_lang('hi')
* wikipedia.languages()
* wikipedia.page('keywords').images[0]
* wikipedia.page('keywords').html()

## Beautiful Soup

In [ ]:
# pip install wikipedia

In [ ]:
# https://kleiber.me/blog/2017/07/22/tutorial-lda-wikipedia/
import pandas as pd
import random
import wikipedia

# rtitles = wikipedia.random(5)

# get 5 Wikipedia page titles based on keywords or manually enter in keywords list
titles = []
keywords = ['ultranationalism', 'religion', 'religious facism', 'state religion', 'deifying rulers']
for key in keywords:
    title = wikipedia.search(key, results=5)
    titles.append(title[0])

# print(titles)
data = []

for title in titles:
    # disambiguous error fix
    try:
        url_title = title.strip().replace(' ', '_')
        url = f'https://en.wikipedia.org/wiki/{url_title}' # left alt, shift, down to duplicate line
        # data.append([title, url, wikipedia.page(title, auto_suggest=False).content, wikipedia.summary(title, auto_suggest=False, sentences=15)])
        data.append([title, url])
    except wikipedia.exceptions.DisambiguationError as e:
        s = random.choice(e.options)
        data.append([title, wikipedia.page(s).content,  wikipedia.summary(title, auto_suggest=False, sentences=15)])

# df = pd.DataFrame(data, columns=['title', 'url', 'content', 'summary'])
pages = pd.DataFrame(data, columns=['title', 'url'])
pages.head()

,title,url
0,Ultranationalism,https://en.wikipedia.org/wiki/Ultranationalism
1,Religion,https://en.wikipedia.org/wiki/Religion
2,Fascism,https://en.wikipedia.org/wiki/Fascism
3,State religion,https://en.wikipedia.org/wiki/State_religion
4,Apotheosis,https://en.wikipedia.org/wiki/Apotheosis


In [ ]:
# wikiscrape
from bs4 import BeautifulSoup
import pandas as pd
import requests

data = []

def make_soup(page):
  # global df
  soup = BeautifulSoup(requests.get(page.url).text)
  s = soup.find_all('h2')
  s_list = [x.get_text().replace('[edit]', '') for x in s]
  # print(pd.Series(s_list))
  data.extend([[page.title, page.url, x.get_text().replace('[edit]', '')] for x in s])

x = pages.apply(make_soup, axis=1)
headings = pd.DataFrame(data, columns=['title', 'url', 'heading'])
drop_list = ['Contents', 'See also', 'References', 'External links', 'Notes', 'Sources', 'Further reading', 'Bibliography']
headings = headings[~headings['heading'].isin(drop_list)]
print(headings.shape)
headings.head()

(33, 3)


,title,url,heading
1,Ultranationalism,https://en.wikipedia.org/wiki/Ultranationalism,Background concepts and broader context
2,Ultranationalism,https://en.wikipedia.org/wiki/Ultranationalism,Historical movements and analysis
3,Ultranationalism,https://en.wikipedia.org/wiki/Ultranationalism,Ultranationalist political parties
4,Ultranationalism,https://en.wikipedia.org/wiki/Ultranationalism,Ultranationalist organizations
5,Ultranationalism,https://en.wikipedia.org/wiki/Ultranationalism,Ultranationalist terrorism


In [ ]:
import re

CLEANR = re.compile('<.*?>')
def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext

data = []
def get_subs(row):
  heading1 = row['heading']
  title = row['title']
  url = row['url']
  soup = BeautifulSoup(requests.get(url).text)
  txt = ''
  txt1 = ''
  target = soup.find('span', attrs={'id': heading1.replace(' ', '_')}).parent
  for sib in target.find_next_siblings():
      if sib.name=='h2':
          break
      else:
          txt += str(sib)
          if sib.name=='p':
            txt1 += str(sib)

  soup2 = BeautifulSoup(txt)
  s = soup2.find_all('h3')
  s_list2 = [x.get_text().replace('[edit]', '') for x in s]
  # print(f'{heading1}\n')
  if len(s_list2) > 0:
    # print(pd.Series(s_list2))
    for i in range(len(s_list2)):
      txt=''
      heading2 = s_list2[i]
      target2 = soup.find('h3', string=heading2)
      target2 = soup.find('span', attrs={'id': heading2.replace(' ', '_')}).parent
      for sib in target2.find_next_siblings():
          if sib.name=='h3':
              break
          else:
            if sib.name=='p':
              txt += sib.text

      data.append([title, url, heading1, heading2, cleanhtml(txt)])
  else:
      data.append([title, url, heading1, 'None', cleanhtml(txt1)])

x = headings.apply(get_subs, axis=1)
df = pd.DataFrame(data, columns=['title', 'url', 'heading', 'subheading', 'txt'])
print(df.shape)
df.head()

(91, 5)


,title,url,heading,subheading,txt
0,Ultranationalism,https://en.wikipedia.org/wiki/Ultranationalism,Background concepts and broader context,None,British political theorist Roger Griffin has s...
1,Ultranationalism,https://en.wikipedia.org/wiki/Ultranationalism,Historical movements and analysis,None,American historian Walter Skya has written in ...
2,Ultranationalism,https://en.wikipedia.org/wiki/Ultranationalism,Ultranationalist political parties,Currently represented in national governments ...,The following political parties have been char...
3,Ultranationalism,https://en.wikipedia.org/wiki/Ultranationalism,Ultranationalist political parties,Represented parties with former ultranationali...,The following political parties historically h...
4,Ultranationalism,https://en.wikipedia.org/wiki/Ultranationalism,Ultranationalist political parties,Formerly represented in national governments o...,Arising out of strident Sri Lankan Tamil natio...


## LDA (Latent Dirichlet Allocation)

In natural language processing, latent Dirichlet allocation (LDA) is a Bayesian network (and, therefore, a generative statistical model) for modeling automatically extracted topics in textual corpora. The LDA is an example of a Bayesian topic model. In this, observations (e.g., words) are collected into documents, and each word's presence is attributable to one of the document's topics. Each document will contain a small number of topics.

Sources:
 * https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation
 * https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer

results = 10
components = 10
topics = 10

vectorizer = TfidfVectorizer(stop_words='english')
vectors = vectorizer.fit_transform(df['txt'].values.astype('U'))

model = LatentDirichletAllocation(n_components=components)
model.fit(vectors)

topics_dictionary = {}
for index, topic in enumerate(model.components_):
    print(f'Topic {index} top words: {[vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-topics:]]}')
    topics_dictionary[index] = [vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-topics:]]



Topic 0 top words: ['ilkhanate', 'world', 'soil', 'culture', 'day', 'art', 'mongol', 'christian', 'dialogue', 'interfaith']
Topic 1 top words: ['include', 'samuel', 'african', 'africa', 'services', 'entitled', 'wrote', 'apotheosis', 'poem', 'mythology']
Topic 2 top words: ['beings', 'putin', 'definition', 'west', '33', 'control', 'sacred', 'dharma', 'case', 'women']
Topic 3 top words: ['apotheose', 'catholic', 'hero', 'lully', 'ultranationalist', 'parties', 'science', 'religiō', 'following', 'apotheosis']
Topic 4 top words: ['factions', 'sponsored', '138', 'yuan', 'tendencies', 'religions', 'confucianism', 'political', 'health', 'religion']
Topic 5 top words: ['global', 'abrahamic', 'follow', 'rates', 'religious', 'population', 'study', 'law', 'judaism', 'wealth']
Topic 6 top words: ['state', 'abolished', 'conquered', 'disappeared', 'fell', 'emperor', 'jurisdictions', 'roman', 'states', 'superstition']
Topic 7 top words: ['299', 'tyranny', '243', 'deification', 'implications', 'worship

In [ ]:
def get_topics(row):
  return ', '.join([top for top in topics_dictionary[row.topic_idx]])

topic_results = model.transform(vectors)
df['topic_idx'] = topic_results.argmax(axis=1)

df['topics']= df.apply(get_topics, axis=1)
df.head()

,title,url,heading,subheading,txt,topic_idx,topics
0,Ultranationalism,https://en.wikipedia.org/wiki/Ultranationalism,Background concepts and broader context,None,British political theorist Roger Griffin has s...,9,"italian, buddhism, political, church, religiou..."
1,Ultranationalism,https://en.wikipedia.org/wiki/Ultranationalism,Historical movements and analysis,None,American historian Walter Skya has written in ...,0,"ilkhanate, world, soil, culture, day, art, mon..."
2,Ultranationalism,https://en.wikipedia.org/wiki/Ultranationalism,Ultranationalist political parties,Currently represented in national governments ...,The following political parties have been char...,3,"apotheose, catholic, hero, lully, ultranationa..."
3,Ultranationalism,https://en.wikipedia.org/wiki/Ultranationalism,Ultranationalist political parties,Represented parties with former ultranationali...,The following political parties historically h...,4,"factions, sponsored, 138, yuan, tendencies, re..."
4,Ultranationalism,https://en.wikipedia.org/wiki/Ultranationalism,Ultranationalist political parties,Formerly represented in national governments o...,Arising out of strident Sri Lankan Tamil natio...,9,"italian, buddhism, political, church, religiou..."


## SpaCy

* https://spacy.io/
* https://medium.com/analytics-vidhya/text-summarization-using-spacy-ca4867c6b744

In [ ]:
# uncomment to download
import spacy.cli

spacy.cli.download('en_core_web_sm')

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


## Language Model and Pipelines

en_core_web_sm

* https://www.kdnuggets.com/2021/03/natural-language-processing-pipelines-explained.html
* https://spacy.io/usage/spacy-101
* https://en.wikipedia.org/wiki/Language_model
* https://builtin.com/data-science/beginners-guide-language-models

In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest

nlp = spacy.load('en_core_web_sm')

In [ ]:
import textwrap
import re

data = []
summary_text = ' '.join([re.sub("\[.*?\]", "", txt) for txt in df.txt])
doc = nlp(summary_text)
keyword = []
stopwords = list(STOP_WORDS)
pos_tag = ['PROPN', 'ADJ', 'NOUN', 'VERB']
for token in doc:
    if(token.text in stopwords or token.text in punctuation):
        continue
    if(token.pos_ in pos_tag):
        keyword.append(token.text)

freq_word = Counter(keyword)
max_freq = Counter(keyword).most_common(1)[0][1]
for word in freq_word.keys():
    freq_word[word] = (freq_word[word]/max_freq)

sent_strength={}
for sent in doc.sents:
    for word in sent:
        if word.text in freq_word.keys():
            if sent in sent_strength.keys():
                sent_strength[sent] += freq_word[word.text]
            else:
                sent_strength[sent] = freq_word[word.text]

    try:
      data.append([sent_strength[sent], str(sent)])
    except:
      pass
    # print(sent)
    # print()

summary = nlargest(10, sent_strength, key=sent_strength.get)
summary = ' '.join([w.text for w in summary])
print(textwrap.fill(summary, 100))
df2 = pd.DataFrame(data, columns=['strength', 'txt'])
df2.sort_values(by=['strength'], ascending=False).head()

A number of disciplines study the phenomenon of religion: theology, comparative religion, history of
religion, evolutionary origin of religions, anthropology of religion, psychology of religion
(including neuroscience of religion and evolutionary psychology of religion), law and religion, and
sociology of religion.  The jurisdictions below give various degrees of recognition in their
constitutions to Eastern Orthodoxy, but without establishing it as the state religion: The following
states recognize some form of Protestantism as their state or official religion: The Anglican Church
of England is the established church in England as well as all three of the Crown Dependencies:
Jurisdictions where a Lutheran church has been fully or partially established as a state recognized
religion include the Nordic States.  Mussolini was aware that Italy did not have the military
capacity to carry out a long war with France or the United Kingdom and waited until France was on
the verge of imminent c

,strength,txt
177,10.305085,A number of disciplines study the phenomenon o...
842,5.864407,The jurisdictions below give various degrees o...
621,5.587571,Mussolini was aware that Italy did not have th...
212,5.457627,"In the field of comparative religion, a common..."
281,5.316384,"In West Africa, these religions include the Ak..."
